<a href="https://colab.research.google.com/github/liranbd1/FaceMaskDetectionLab/blob/ok/MTCNN_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Missing API's

* facenet-pytroch library for MTCNN

In [ ]:
!pip install facenet-pytorch

#Connecting to GDrive
All of the dataset and saving of the models are stored in GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

#Imports

In [ ]:
import pandas as pd
import os
from facenet_pytorch.models.mtcnn import MTCNN
from PIL import Image
import torchvision
from torchvision import transforms
from IPython.core.display import Math
from torch.utils.data import Dataset
import torch

#Hyper Params

In [ ]:
root = "/content/gdrive/MyDrive/MaskDetection" # Root folder in GDrive to the data

device = 'cude' if torch.cuda.is_available() else 'cpu'

images_path = os.path.join(root, 'images')
csv_path = os.path.join(root,'train.csv')
mtcnn_model = MTCNN(keep_all = True).to(device)

In [ ]:
def process_df(csv_path):
  df = pd.read_csv(csv_path)
  df = df.drop(df[(df['classname'] != 'face_other_covering') & (df['classname'] != 'face_no_mask') & (df['classname'] != 'face_with_mask') & (df['classname'] != 'face_with_mask_incorrect')].index) # This section will remove all the rows which their classname is not one of the mentiond here.
  df.loc[df['classname'] == 'face_with_mask_incorrect', 'classname'] = 'face_with_mask' # Replace all face_with_mask_incorrect with face_with_mask
  df.loc[df['classname'] == 'face_other_covering', 'classname'] = 'face_no_mask' # Replace all face_other_covering with face_with_no_mask
  return df

#Intersection over union util

Give two bounding boxes we check to see how much of they intersect each other.

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

# Validate sample util 
Making sure we don't get any uneeded data due to problems with the dataset

In [ ]:
def validate_sample(sample_list):
  cleaned = list(filter(lambda c: c[1] != -1, sample_list))
  filtered=set([match[1] for match in cleaned])
  
  if(len(filtered) != len(cleaned)):
    # print('Error' + '^^'*10)
    return False

  return True

In [ ]:
img_dataframe = process_df(csv_path)

# Gathering data on the MTCNN ability to detect faces.


In [ ]:
unique_img_names = image_dataframe['name'].unique()

match_sample_idx = []
bad_img = []
idx_found = [] #This list can be initialized one and just add to it since the idx that is saved is of the img row, so it will be unique for each row.

'''
For each bounding box that the MTCNN finds, check if there is a match of some 
correlation with the dataset bounding boxes (bbox and sample_box respectivly)

Every match we get is True Positive.
False Negative is the differance between the total rows and TP.
If we got a bbox but didn't find a match of a sample_box we consider it as 
MTCNN found something that is not there meaning False Positive and it will be 
annotated with max_idx = -1 and max_iou = 0
'''

for img_name in unique_img_names:
  tmp_list = []
  img = Image.open(os.path.join(image_path, img_name))
  bboxes, probs = mtcnn.detect(img)
  subset = df.loc[df['name'] == img_name]

  if bboxes is not None:
    for bidx, mtcnn_data in enumerate(zip(bboxes,probs)):
      bbox = mtcnn_data[0]
      prob = mtcnn_data[1]
      max_iou = 0
      max_idx = -1 

      for idx, sample in subset.iterrows():
        # For some reason the dataset defined X as a set of two cordinates and Y as another so X1 X2 is two cordinates for the same point in 2D space
        sample_bbox = [sample['x1'], sample['x2'], sample['y1'], sample['y2']]
        iou = bb_intersection_over_union(sample_bbox, bbox)
        
        if iou>max_iou and idx not in idx_found:
          max_iou = iou
          max_idx = idx

      idx_found.append(max_idx)
      tmp_list.append([img_name, max_idx, max_iou, prob])

  if len(tmp_list) != 0:
    if not validate_sample(tmp_list):
      bad_img.append(img_name)
    if len(tmp_list[0]) != 0:
      match_sample_idx.append(tmp_list)


In [ ]:
TP = sum([1 for match in match_sample_idx if match[1]!= -1])
FN = len(image_dataframe) - TP
FP = len(match_sample_idx) - TP

# Code to create the graphs we want


# Here is an example to how save object to GDrive

I used this to save matplotlib figures, all the important part is to make sure the path you want exist and if not create it (lines 3 to 8) 

Then we can use any built in method to save objects in python (In here you can see how matplotlib is saving the plots as png images

In [ ]:
def plot_report(x_axis, title, y_axis):
  file_name = title
  if not os.path.exists('./gdrive/MyDrive/Deep_Learning_HW2/plots'):
    try: # Since we can throw an error here we prefer to wrap it in try and catch and raise the error back to not continue the run
      !mkdir './gdrive/MyDrive/Deep_Learning_HW2/plots'
    except OSError as error:
      print(error)
      raise error 

  fig = plt.figure()
  plt.plot(x_axis, y_axis, 'b')
  plt.title(title)
  file_path = os.path.join('./gdrive/MyDrive/Deep_Learning_HW2/plots', f'{file_name}')
  fig.savefig(file_path, format='png')

# Saving the model in some path

In [ ]:
torch.save(model.state_dict(), path)